# ADS 509 Module 1: APIs and Web Scraping
Shailja Somani\
ADS 509 Summer 2024\
May 13, 2024

This notebook has two parts. In the first part, you will scrape lyrics from AZLyrics.com. In the second part, you'll run code that verifies the completeness of your data pull. 

For this assignment you have chosen two musical artists who have at least 20 songs with lyrics on AZLyrics.com. We start with pulling some information and analyzing them.


# Importing Libraries

In [57]:
import os
import datetime
import re

# for the lyrics scrape section
import requests
import time
from bs4 import BeautifulSoup
from collections import defaultdict, Counter
import random

In [62]:
# Use this cell for any import statements you add
import shutil
from bs4 import Comment

---

# Lyrics Scrape

This section asks you to pull data by scraping www.AZLyrics.com. In the notebooks where you do that work you are asked to store the data in specific ways. 

In [2]:
# we'll use this dictionary to hold both the artist name and the link on AZlyrics
# Added the two artists of my choosing
artists = {'kelly':"https://www.azlyrics.com/r/rowland.html",
           'usher':"https://www.azlyrics.com/u/usher.html"} 

## A Note on Rate Limiting

The lyrics site, www.azlyrics.com, does not have an explicit maximum on number of requests in any one time, but in our testing it appears that too many requests in too short a time will cause the site to stop returning lyrics pages. (Entertainingly, the page that gets returned seems to only have the song title to [a Tom Jones song](https://www.azlyrics.com/lyrics/tomjones/itsnotunusual.html).) 

Whenever you call `requests.get` to retrieve a page, put a `time.sleep(5 + 10*random.random())` on the next line. This will help you not to get blocked. If you _do_ get blocked, which you can identify if the returned pages are not correct, just request a lyrics page through your browser. You'll be asked to perform a CAPTCHA and then your requests should start working again. 

## Part 1: Finding Links to Songs Lyrics

That general artist page has a list of all songs for that artist with links to the individual song pages. 

Q: Take a look at the `robots.txt` page on www.azlyrics.com. (You can read more about these pages [here](https://developers.google.com/search/docs/advanced/robots/intro).) Is the scraping we are about to do allowed or disallowed by this page? How do you know? 

A: The robots.txt page on www.azlyrics.com states the following:\
`User-agent: * `\
`Disallow: /lyricsdb/ `\
`Disallow: /song/ `\
`Allow: /`

`User-agent: 008 `\
`Disallow: / `

The above means that for all users except user agent 008 (a specific user that AZLyrics has identified as potentially suspicious, a scammer, etc and is thus not allowed to scrape anything from the site) are not allowed to scrape from URLs that contain `/lyricsdb/` or `/song/`. However, users are permitted to scrape from all other pages on the website. We will be scraping from pages that begin with `www.azlyrics.com/<letter>/` (for the artist pages) and `www.azlyrics.com/lyrics/` (for the lyrics pages), so both should be allowed.

In [ ]:
# (Cleared the output for this after running so PDF would not be super long)
# Let's set up a dictionary of lists to hold our links
lyrics_pages = defaultdict(list)

for artist, artist_page in artists.items():
    # request the page and sleep
    r = requests.get(artist_page)
    time.sleep(5 + 10*random.random())
    
    # Use BeautifulSoup to parse
    artist_pg_soup = BeautifulSoup(r.text, 'html.parser')

    # Investigated artist_pg_soup output in test code below to understand how lyrics page links are stored 
    # Extract all raw links
    links_raw = artist_pg_soup.find_all('a', href=True)
    
    # Loop through links to format correctly & put in dict where the key is the artist and the
        # value is a list of links
    for link in links_raw:
        href_raw = link['href']
        # Check if lyric link
        if "/lyrics/" in href_raw:
            # Check if URL is full path
            if href_raw.startswith('http'):
                complete_link = href_raw
            else: # Complete URL if not full path
                complete_link = 'https://www.azlyrics.com' + href_raw
            lyrics_pages[artist].append(complete_link)

# Print dict to check
lyrics_pages # Checked, then cleared output so not messy in PDF generated 

Let's make sure we have enough lyrics pages to scrape. 

In [5]:
for artist, lp in lyrics_pages.items() :
    assert(len(set(lp)) > 20) 

In [6]:
# Let's see how long it's going to take to pull these lyrics 
# if we're waiting `5 + 10*random.random()` seconds 
for artist, links in lyrics_pages.items() : 
    print(f"For {artist} we have {len(links)}.")
    print(f"The full pull will take for this artist will take {round(len(links)*10/3600,2)} hours.")

For kelly we have 105.
The full pull will take for this artist will take 0.29 hours.
For usher we have 225.
The full pull will take for this artist will take 0.62 hours.


## Part 2: Pulling Lyrics

Now that we have the links to our lyrics pages, let's go scrape them! Here are the steps for this part. 

1. Create an empty folder in our repo called "lyrics". 
1. Iterate over the artists in `lyrics_pages`. 
1. Create a subfolder in lyrics with the artist's name. For instance, if the artist was Cher you'd have `lyrics/cher/` in your repo.
1. Iterate over the pages. 
1. Request the page and extract the lyrics from the returned HTML file using BeautifulSoup.
1. Use the function below, `generate_filename_from_link`, to create a filename based on the lyrics page, then write the lyrics to a text file with that name. 


In [59]:
# I made some changes to this definition to make it cleaner - explained in comments marked with my initials (SS)
def generate_filename_from_link(link) :
    if not link :
        return None
    
    # drop the http or https and the html
    # (SS) Added :// to what is being removed here so can later split on first /
    name = link.replace("https://","").replace("http://","")  
    name = name.replace(".html","") # (SS) Fixed this - said "link.replace", should be "name.replace"

    name = name.replace("www.azlyrics.com/lyrics/","") # (SS) Add the full URL before lyrics here to clean up more
    
    # (SS) Remove artist name since will already be in artist subfolder
    name = name.split('/', 1)[1]
    
    # Replace useless chareacters with UNDERSCORE
    name = name.replace(".","_").replace("/","_")
    
    # tack on .txt
    name = name + ".txt"
    
    return(name)

In [71]:
# Make the lyrics folder here. If you'd like to practice your programming, add functionality 
# that checks to see if the folder exists. If it does, then use shutil.rmtree to remove it and create a new one.

# Turned into function so can also use to create artist subfolders
def make_lyrics_folder(folder_name):
    if folder_name != 'lyrics':
        folder_name = 'lyrics/' + folder_name
    if os.path.isdir(folder_name): 
        shutil.rmtree(folder_name)
    os.mkdir(folder_name)
    
# Actually create lyrics folder
make_lyrics_folder('lyrics')

In [80]:
#url_stub = "https://www.azlyrics.com" - don't believe this is needed
start = time.time()

for artist in lyrics_pages:
    # Move page counter to within this loop so resets for each artist, but not for each page within artist
    total_pages = 0 

    # Use this space to carry out the following steps: 
    
    # 1. Build a subfolder for the artist
    make_lyrics_folder(artist)
    # 2. Iterate over the lyrics pages
    for page in lyrics_pages.get(artist):
        # 3. Request the lyrics page. 
        # Don't forget to add a line like `time.sleep(5 + 10*random.random())`
        # to sleep after making the request
        r = requests.get(page)
        time.sleep(5 + 10*random.random())

        # Use BeautifulSoup to parse
        lyric_pg_soup = BeautifulSoup(r.text, 'html.parser')
        
        # 4. Extract the title 
        title_raw = lyric_pg_soup.find('h1')
        title = title_raw.text.replace('" lyrics', '').replace('"', '').strip()
        
        # 4.5. Extract the lyrics
        comment = lyric_pg_soup.find(string=lambda text: isinstance(text, Comment) and 
                                 "Usage of azlyrics.com content by any third-party lyrics provider is prohibited" in text)
        # If the comment exists, get the parent div of the comment
        if comment:
            parent_div = comment.find_parent('div')  
            # If parent div exists (error handling), get all text within it with some minor cleaning
            if parent_div:
                lyrics = []
                for elem in parent_div.children:
                    if elem.name == 'br':
                        lyrics.append('\n')
                    elif isinstance(elem, str):
                        lyrics.append(elem.strip())
                lyrics = ''.join(lyrics).strip()
        # Remove comment from output so just actual lyrics
        lyrics = lyrics.replace("Usage of azlyrics.com content by any third-party lyrics provider is prohibited by our licensing agreement. Sorry about that.", "")

        # 5. Write out the title, two returns ('\n'), and the lyrics. Use `generate_filename_from_link`
        #    to generate the filename. 
        filename = generate_filename_from_link(page)
        # Put within subfolder created for artist 
        with open(os.path.join('lyrics', artist, filename), 'w', encoding='utf-8') as file:
            file.write(title + '\n\n' + lyrics)
    
        # Remember to pull at least 20 songs per artist. It may be fun to pull all the songs for the artist
        total_pages += 1
        # Break if total_pages == 20 to save time
        if total_pages == 20: 
            break 
    

In [81]:
print(f"Total run time was {round((time.time() - start)/3600,2)} hours.")

Total run time was 0.12 hours.


---

# Evaluation

This assignment asks you to pull data by scraping www.AZLyrics.com.  After you have finished the above sections , run all the cells in this notebook. Print this to PDF and submit it, per the instructions.

In [74]:
# Simple word extractor from Peter Norvig: https://norvig.com/spell-correct.html
def words(text): 
    return re.findall(r'\w+', text.lower())

## Checking Lyrics 

The output from your lyrics scrape should be stored in files located in this path from the directory:
`/lyrics/[Artist Name]/[filename from URL]`. This code summarizes the information at a high level to help the instructor evaluate your work. 

In [82]:
artist_folders = os.listdir("lyrics/")
artist_folders = [f for f in artist_folders if os.path.isdir("lyrics/" + f)]

for artist in artist_folders : 
    artist_files = os.listdir("lyrics/" + artist)
    artist_files = [f for f in artist_files if 'txt' in f or 'csv' in f or 'tsv' in f]

    print(f"For {artist} we have {len(artist_files)} files.")

    artist_words = []

    for f_name in artist_files : 
        with open("lyrics/" + artist + "/" + f_name) as infile : 
            artist_words.extend(words(infile.read()))

            
    print(f"For {artist} we have roughly {len(artist_words)} words, {len(set(artist_words))} are unique.")


For kelly we have 20 files.
For kelly we have roughly 9321 words, 1091 are unique.
For usher we have 20 files.
For usher we have roughly 9245 words, 763 are unique.


## Test Code
*Note:* I would not have this section in Production code, but I am leaving it in this assignment submission for the sake of showing my work/thought process for the above code.

In [83]:
# Investigate full output from BeautifulSoup handling of artist page so know how to parse out links
artist_pg_soup # output cleared so as not to make PDF very long

# Output result: lyric links are stored such as <a href="/lyrics/usher/loveemall.html"

In [16]:
# Test code from within generate_filename_from_link function to see what intermediate outputs are
link = "https://www.azlyrics.com/lyrics/usher/illmakeitright.html"
name = link.replace("https","").replace("http","")
name = name.replace(".html","") # (SS) Fixed this - said "link.replace", should be "name.replace"

name = name.replace("www.azlyrics.com/lyrics/","")
name

'://usher/illmakeitright'

In [18]:
# Test generate_filename_from_link function
generate_filename_from_link("https://www.azlyrics.com/lyrics/usher/illmakeitright.html")

'illmakeitright.txt'

In [ ]:
# Test reading lyrics 
artist = 'kelly'
artist_page = lyrics_pages.get(artist)[0]

r = requests.get(artist_page)
time.sleep(5 + 10*random.random())

# Use BeautifulSoup to parse
lyric_pg_soup = BeautifulSoup(r.text, 'html.parser')
lyric_pg_soup # output cleared so not super long PDF 

In [27]:
# Extract title test
title_raw = lyric_pg_soup.find('h1')
title = title_raw.text.replace('" lyrics', '').replace('"', '').strip()
title

'Stole'

In [51]:
# Extract lyrics test - using Usage of azlyrics.com comment
comment = lyric_pg_soup.find(string=lambda text: isinstance(text, Comment) and 
                                 "Usage of azlyrics.com content by any third-party lyrics provider is prohibited" in text)
if comment:
    parent_div = comment.find_parent('div')  # Get the parent div of the comment
    if parent_div:
        lyrics = []
        for elem in parent_div.children:
            if elem.name == 'br':
                lyrics.append('\n')
            elif isinstance(elem, str):
                lyrics.append(elem.strip())
        lyrics = ''.join(lyrics).strip()
        
# Remove comment from output
lyrics = lyrics.replace("Usage of azlyrics.com content by any third-party lyrics provider is prohibited by our licensing agreement. Sorry about that.", "")

In [52]:
lyrics

"He was always such a nice boy\nThe quiet one, with good intentions\nHe was down for his brother, respectful to his mother\nA good boy\nBut good don't get attention\nOne kid with a promise\nThe brightest kid in school, he's not a fool\nReadin' books about science and smart stuff\nIt's not enough, no\n'Cause smart don't make you cool, whoa\n\nHe's not invisible anymore\nWith his Father's nine and a broken fuse\nSince he walked through that classroom door\nHe's all over prime-time news\n\nMary's got the same size hands\nAs Marilyn Monroe\nShe put her fingers in the imprints\nAt Mann's Chinese Theater Show\nShe could've been a movie star\nNever got the chance to go that far\nHer life was stole, oh\nNow we'll never know\n(No, no, no, no, oh)\n\nThey were cryin' to the camera, said he never fitted in\nHe wasn't welcome\nHe showed up to the parties we was hangin' in\nSome guys were puttin' him down\nBullyin' him 'round, round\nNow I wish I would've talked to him\nGave him the time of day and

In [58]:
# Test writing to file
make_lyrics_folder(artist)
filename = generate_filename_from_link(artist_page)
with open(os.path.join(artist, filename), 'w', encoding='utf-8') as file:
    file.write(title + '\n\n' + lyrics)